 <img src="../images/logos/Ideam_logo.png" width=300 alt="Ideam_Logo"></img> 

# Acceso a información hidrometeorológica en Colombia - Estaciones

---

## Resumen
En este cuadernillo (Notebook) aprenderemos:

1. Introduccion a la red de monitoreo del IDEAM
2. Cátalogo de estaciones de IDEAM
3. Consulta de datos usando la plataforma [datosabiertos.gov.co](https://www.datos.gov.co/) 
1. Consulta de datos de temperatura y precipitación
1. Otros datos disponibles

## Prerequisitos
| Conceptos | Importancia | Notas |
| --- | --- | --- |
| [Introducción a Pandas](https://foundations.projectpythia.org/core/pandas.html) | Necesario | lectura de datos tabulares |
| [Introducción a Datetime](https://foundations.projectpythia.org/core/datetime/datetime.html) | Necesario | Entender estampas de tiempo |

- **Tiempo de aprendizaje**: 30 minutos

# 1. Catalogo nacional de estaciones de IDEAM 

El ideam cuenta con .....

## Librerias


In [ ]:
import pandas as pd
# import folium
import cartopy.crs as ccrs
import cartopy.feature as feature
import matplotlib.pyplot as plt

## Acceso al catalog en [datosabiertos.gov.co](https://www.datos.gov.co/) 

El catalogo nacional de estaciones de IDEAM se encuentra disponible en el siguiente link. Podemos leer el cat[alogo usando `pandas`

In [ ]:
df_cat = pd.read_excel('http://bart.ideam.gov.co/cneideam/CNE_IDEAM.xls')

In [ ]:
#df_cat.head()

In [ ]:
df_cat.info()

## Mapa de estaciones 

Podemos usar `cartopy` para hacer un mapa y visualizar las estaciones de monitoreo en el pais

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()}, dpi=150)
ax.coastlines()
gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree())
ax.scatter(df_cat['longitud'], df_cat['latitud'], transform=ccrs.PlateCarree(), s=.5)
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN)
ax.add_feature(feature.COASTLINE, linewidth=.5)
ax.add_feature(feature.BORDERS, linewidth=.5)

podemos agrupar la data por área operativa, tipo de estacion, tecnologia, y otras variables

In [ ]:
df_grp = df_cat.groupby('AREA_OPERATIVA')
# df_grp = df_cat.groupby('TECNOLOGIA')
# df_grp = df_cat.groupby('ESTADO')

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()}, dpi=150)

for _, group in df_grp:
    ax.scatter(group['longitud'], group['latitud'], transform=ccrs.PlateCarree(), s=.5, label=_)
    
ax.coastlines()
gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree())
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN)
ax.add_feature(feature.COASTLINE, linewidth=.5)
ax.add_feature(feature.BORDERS, linewidth=.5)
# ax.legend(fontsize=5)

Podemos validar el numero total de estaciones activas, en matenimiento y suspendidas de acuerdo con la información contenida en el catálogo

In [ ]:
for grp in df_grp.groups.keys():
    print(f"{grp}: {len(df_grp.get_group(grp))}")

También podemos hacer mapas interactivos usando [folium](https://python-visualization.github.io/folium/latest/)

In [ ]:
import folium
from folium import plugins
from folium.plugins import MarkerCluster

In [ ]:
min_lon, max_lon, min_lat, max_lat = -90, -72, -1, 14

map_ = folium.Map(location=[8, -76],
                  zoom_start = 6, 
                  min_lat=min_lat, 
                  max_lat=max_lat, 
                  min_lon=min_lon, 
                  max_lon=max_lon, 
                  zoom_control=False,
                  control_scale=True,
                  scrollWheelZoom=True,
                  width=1000,height=600)
marker_cluster = MarkerCluster(name="Estaciones").add_to(map_)

folium.TileLayer('cartodbpositron').add_to(map_)
folium.TileLayer('openstreetmap').add_to(map_)
folium.TileLayer('stamenterrain').add_to(map_)
folium.TileLayer('cartodbdark_matter').add_to(map_)
folium.LayerControl().add_to(map_)

minimap = plugins.MiniMap()
_ = map_.add_child(minimap)
# _

Ahora agregamos las estaciones usando la siguiente función

In [ ]:
def plot_station(row):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.Marker(location=[row.latitud, row.longitud], popup=row["CODIGO"]).add_to(marker_cluster)

In [ ]:
df_cat.apply(plot_station, axis=1)
map_

## Acceso a la informacion historica de IDEAM usando [datosabiertos.gov.co](https://www.datos.gov.co/)

la información historica de multiples sensores se puede consultar atraves de la plataforma de datos abiertos usando el aplicativo [sodapy](https://dev.socrata.com/). Socrata utiliza un módulo denominado `Socrata` que permite realizar consultas al repositorio. Cada variable hidrometeorógica dispuesta se puede consultar usando el su respectivo código del set de datos.


| Variable | Código del set de datos |
| --- | --- |
| [Dirección del viento](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Direcci-n-Viento/kiw7-v9ta) | kiw7-v9ta |
| [Nivel instantaneo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-Instant-neo/bdmn-sqnh) | bdmn-sqnh |
| [Temperatura Minima del Aire](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Temperatura-M-nima-del-Aire/afdg-3zpb) | afdg-3zpb |
| [Temperatura Maxima del Aire](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Temperatura-M-xima-del-Aire/ccvq-rp9s) | ccvq-rp9s |
| [Velocidad del Viento](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Velocidad-Viento/sgfv-3yp8) | sgfv-3yp8 |
| [Nivel Maximo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-M-ximo/vfth-yucv) | vfth-yucv |
| [Nivel Minimo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-M-nimo/pt9a-aamx) | pt9a-aamx |
| [Humedad del Aire](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Humedad-del-Aire-2-metros/uext-mhny) | uext-mhny |
| [Temperatura](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Datos-Hidrometeorol-gicos-Crudos-Red-de-Estaciones/sbwg-7ju4) | sbwg-7ju4 |
| [Nivel del mar mínimo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-del-Mar-M-nimo/7z6g-yx9q) | 7z6g-yx9q |
| [Nivel del mar máximo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-del-Mar-M-ximo/uxy3-jchf) | uxy3-jchf |
| [Nivel del mar](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-del-Mar/ia8x-22em) | ia8x-22em |
| [Presión Atmosferica](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Presi-n-Atmosf-rica/62tk-nxj5) | 62tk-nxj5 |
| [Precipitación](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Precipitaci-n/s54a-sgyg) | s54a-sgyg |

In [ ]:
# importamos la libreria Socrata
from sodapy import Socrata

### Precipitación (s54a-sgyg)

Vamos a consultar los datos de `precipitación` reportada en la página. por ende vamos a usar el código `s54a-sgyg`. Para esto usamos el metodo `Socrata`, pasamos la direccion del repositorio y `None` que corresponde a la no autenticación

In [ ]:
# conexión cliente usando socrata al repositorio de datos abiertos
client = Socrata("www.datos.gov.co", None)

Una vez creado el cliente empezamos a hacer la consulta de datos  usando `client.get` y pasando los respectivos parámetros `dataset_identifier`, de la tabla anterior , y `limit` para generar consultas no muy grandes para efectos demostrativos. El resultado es una lista con multiples diccionarios como se puede ver a continuación.

In [ ]:
# Solicitud de informacion al repositorio de interés
results = client.get(dataset_identifier="s54a-sgyg", limit=2000)
results[:1]

Estos resultados los podemos convertir en un `Dataframe` usando [pandas.Dataframe.from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html)

In [ ]:
results_df = pd.DataFrame.from_records(results)
results_df.head()

Ahora podemos usar filtrar los datos por diferentes campos como el `codigoestacion`, `fechaobservacion`, o `valorobservado`. Podemos pasar parametros `SQL` como `where`, `AND`, `IN`, entre otros, en el método `client.get`

In [ ]:
# client.get?

In [ ]:
# Solicitud de informacion para la estación de la Universidad Nacional - Bogotá - 0021205012
ppt_query = client.get(dataset_identifier="s54a-sgyg", 
                       select="fechaobservacion, valorobservado, codigoestacion", 
                       where="codigoestacion IN ('0021205012') \
                              AND fechaobservacion > '2017'")
df_est = pd.DataFrame.from_records(ppt_query)
df_est.head()

In [ ]:
# Solicitud de informacion para la estación de la Universidad Nacional - Bogotá - 0021205012
ppt_query = client.get(dataset_identifier="s54a-sgyg", 
                       select="fechaobservacion, valorobservado, codigoestacion, nombreestacion", 
                       where="fechaobservacion > '2023'")
df_est = pd.DataFrame.from_records(ppt_query)
df_est.head()

#### Gráfico de la serie temporal 
Podemos generar una serie temporal usando la información resultado de la consulta. Sin embargo, primero debemos revisar el tipo de dato resulta

In [ ]:
df_est.info()

In [ ]:
df_est.index = pd.to_datetime(df_est['fechaobservacion'])
df_est.valorobservado = df_est['valorobservado'].astype(float)
df_est = df_est.sort_index()
df_est.tail()

In [ ]:
# pd.options.plotting.backend = 'holoviews'
fig, ax = plt.subplots(figsize=(12, 3))
df_est['valorobservado'].plot(ax=ax)

### Temperatura (sbwg-7ju4)

De manera similar podemos consultar otros registros como los de temperatura. Cambiamos el identificador de set de datos y generamos una nueva consulta

In [ ]:
# Solicitud de informacion para la estación de la Universidad Nacional - Bogotá - 0021205012
temp_query = client.get(dataset_identifier="sbwg-7ju4", 
                       select="fechaobservacion, valorobservado, codigoestacion", 
                       where="codigoestacion IN ('0021205012') \
                              AND fechaobservacion > '2017'")
df_temp = pd.DataFrame.from_records(temp_query)
df_temp.index = pd.to_datetime(df_temp['fechaobservacion'])
df_temp.valorobservado = df_temp['valorobservado'].astype(float)
df_temp = df_temp.sort_index()
df_temp.tail()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 3))
df_temp['valorobservado'].plot(c='C00', lw=0.5, ax=ax)

---

## Resumen final
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!